In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb09 = pd.read_csv(path+'../sub/lgCV_0906.csv.gz', compression='gzip')
lgb10 = pd.read_csv(path+'../sub/lgCV_1006.csv.gz', compression='gzip')
lgb11A= pd.read_csv(path+'../sub/lgCV_1106A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [2]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb09.rename(columns={'deal_probability': 'lgb09_preds' }, inplace=True)
lgb10.rename(columns={'deal_probability': 'lgb10_preds' }, inplace=True)
lgb11A.rename(columns={'deal_probability': 'lgb11A_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb09, on='item_id')\
                .merge(lgb10, on='item_id')\
                .merge(lgb11A, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [3]:
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

['rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11A_preds',
 'lgb02A_preds',
 'rnn27_preds']

In [4]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['max'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0)

In [5]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = preds_df[p2] - preds_df[p1]

difference_rnn_preds__mlp_preds
difference_rnn_preds__lgb31_preds
difference_rnn_preds__lgb09_preds
difference_rnn_preds__lgb10_preds
difference_rnn_preds__lgb11A_preds
difference_rnn_preds__lgb02A_preds
difference_rnn_preds__rnn27_preds
difference_mlp_preds__lgb31_preds
difference_mlp_preds__lgb09_preds
difference_mlp_preds__lgb10_preds
difference_mlp_preds__lgb11A_preds
difference_mlp_preds__lgb02A_preds
difference_mlp_preds__rnn27_preds
difference_lgb31_preds__lgb09_preds
difference_lgb31_preds__lgb10_preds
difference_lgb31_preds__lgb11A_preds
difference_lgb31_preds__lgb02A_preds
difference_lgb31_preds__rnn27_preds
difference_lgb09_preds__lgb10_preds
difference_lgb09_preds__lgb11A_preds
difference_lgb09_preds__lgb02A_preds
difference_lgb09_preds__rnn27_preds
difference_lgb10_preds__lgb11A_preds
difference_lgb10_preds__lgb02A_preds
difference_lgb10_preds__rnn27_preds
difference_lgb11A_preds__lgb02A_preds
difference_lgb11A_preds__rnn27_preds
difference_lgb02A_preds__rnn27_preds


In [6]:
gc.collect()

286

In [7]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb02A_preds,...,difference_lgb09_preds__lgb10_preds,difference_lgb09_preds__lgb11A_preds,difference_lgb09_preds__lgb02A_preds,difference_lgb09_preds__rnn27_preds,difference_lgb10_preds__lgb11A_preds,difference_lgb10_preds__lgb02A_preds,difference_lgb10_preds__rnn27_preds,difference_lgb11A_preds__lgb02A_preds,difference_lgb11A_preds__rnn27_preds,difference_lgb02A_preds__rnn27_preds
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.109624,0.107764,0.095101,0.109617,...,-0.001860,-0.014523,-0.000007,-0.041293,-0.012663,0.001853,-0.039433,0.014517,-0.026770,-0.041287
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.157466,0.161107,0.140269,0.160045,...,0.003640,-0.017198,0.002578,-0.086828,-0.020838,-0.001062,-0.090469,0.019776,-0.069630,-0.089406
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.245236,0.219107,0.234076,0.241336,...,-0.026129,-0.011160,-0.003900,-0.083311,0.014969,0.022229,-0.057183,0.007260,-0.072151,-0.079412
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.289101,0.255519,0.241802,0.336456,...,-0.033582,-0.047299,0.047355,-0.010297,-0.013717,0.080937,0.023285,0.094654,0.037002,-0.057652
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.511145,0.511676,0.504849,0.421989,...,0.000532,-0.006295,-0.089156,-0.160860,-0.006827,-0.089688,-0.161392,-0.082861,-0.154565,-0.071704


### Train scores

In [8]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) and ('difference' not in col)]
pred_cols

['lgb27_preds',
 'rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb02_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11A_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'preds_sum']

In [9]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb09_preds:  0.21581543265040168
RMSE lgb10_preds:  0.21533498019117664
RMSE lgb11A_preds:  0.21403812331071156
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608


In [10]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [11]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb09_preds,0.215514,0.215678,0.218418,0.216073,0.212293
6,lgb10_preds,0.215240,0.215533,0.217072,0.215802,0.212023
7,lgb11A_preds,0.215011,0.215205,0.211879,0.215771,0.211735
8,lgb02A_preds,0.216081,0.216051,0.218825,0.216562,0.212741
9,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379


### Correlations in test and train

In [12]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.978331,0.978298,0.966819,0.983143,0.916804
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.908867,0.913286,0.915655,0.911443,0.985360
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.909521,0.913883,0.915367,0.913600,0.925010
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.981891,0.981375,0.966656,0.986851,0.914204
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.982596,0.982052,0.970858,0.987434,0.916726
lgb09_preds,0.978331,0.908867,0.909521,0.981891,0.982596,1.000000,0.990447,0.976755,0.988708,0.912375
lgb10_preds,0.978298,0.913286,0.913883,0.981375,0.982052,0.990447,1.000000,0.982841,0.985858,0.916436
lgb11A_preds,0.966819,0.915655,0.915367,0.966656,0.970858,0.976755,0.982841,1.000000,0.971893,0.918727
lgb02A_preds,0.983143,0.911443,0.913600,0.986851,0.987434,0.988708,0.985858,0.971893,1.000000,0.914911
rnn27_preds,0.916804,0.985360,0.925010,0.914204,0.916726,0.912375,0.916436,0.918727,0.914911,1.000000


In [13]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb02A_preds,rnn27_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.979729,0.979341,0.975840,0.984742,0.930692
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.926158,0.928367,0.927119,0.928496,0.988198
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.918064,0.920496,0.919275,0.922121,0.935069
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.982874,0.982519,0.978877,0.988043,0.931468
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.984427,0.983525,0.980105,0.989577,0.931447
lgb09_preds,0.979729,0.926158,0.918064,0.982874,0.984427,1.000000,0.991413,0.988267,0.989106,0.929748
lgb10_preds,0.979341,0.928367,0.920496,0.982519,0.983525,0.991413,1.000000,0.992340,0.986654,0.931841
lgb11A_preds,0.975840,0.927119,0.919275,0.978877,0.980105,0.988267,0.992340,1.000000,0.983556,0.930518
lgb02A_preds,0.984742,0.928496,0.922121,0.988043,0.989577,0.989106,0.986654,0.983556,1.000000,0.932071
rnn27_preds,0.930692,0.988198,0.935069,0.931468,0.931447,0.929748,0.931841,0.930518,0.932071,1.000000


In [14]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
cols = list(set(cols))

In [15]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [16]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [17]:
len(train_X)

1202739

In [18]:
%%time
gc.collect()
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# [1846]	training's rmse: 0.210355	valid_1's rmse: 0.213296

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219869	valid_1's rmse: 0.220256
[200]	training's rmse: 0.213441	valid_1's rmse: 0.2141
[300]	training's rmse: 0.212261	valid_1's rmse: 0.213158
[400]	training's rmse: 0.211883	valid_1's rmse: 0.212981
[500]	training's rmse: 0.211657	valid_1's rmse: 0.212931
[600]	training's rmse: 0.211474	valid_1's rmse: 0.212908
[700]	training's rmse: 0.211315	valid_1's rmse: 0.212898
[800]	training's rmse: 0.211175	valid_1's rmse: 0.212889
[900]	training's rmse: 0.211038	valid_1's rmse: 0.21288
[1000]	training's rmse: 0.21091	valid_1's rmse: 0.212873
[1100]	training's rmse: 0.210784	valid_1's rmse: 0.212869
[1200]	training's rmse: 0.210661	valid_1's rmse: 0.212865
[1300]	training's rmse: 0.21054	valid_1's rmse: 0.212861
[1400]	training's rmse: 0.210418	valid_1's rmse: 0.212858
[1500]	training's rmse: 0.210301	valid_1's rmse: 0.212856
[1600]	training's rmse: 0.210191	valid_1's rmse: 0.212855
Early stopping, best ite

In [19]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(7289, 'param_1'),
 (4346, 'param_2'),
 (3615, 'region'),
 (2005, 'category_name'),
 (1740, 'lgb11A_preds'),
 (1560, 'rnn27_preds'),
 (1351, 'difference_rnn_preds__rnn27_preds'),
 (1238, 'mlp_preds'),
 (1237, 'avg'),
 (1075, 'difference_lgb31_preds__lgb11A_preds'),
 (1015, 'min'),
 (899, 'difference_lgb31_preds__lgb09_preds'),
 (883, 'difference_lgb10_preds__lgb11A_preds'),
 (837, 'difference_lgb11A_preds__lgb02A_preds'),
 (803, 'difference_lgb09_preds__lgb02A_preds'),
 (760, 'difference_rnn_preds__mlp_preds'),
 (748, 'difference_mlp_preds__rnn27_preds'),
 (732, 'difference_lgb31_preds__lgb10_preds'),
 (698, 'price'),
 (693, 'difference_lgb10_preds__lgb02A_preds'),
 (678, 'difference_lgb09_preds__lgb10_preds'),
 (658, 'difference_rnn_preds__lgb11A_preds'),
 (653, 'difference_lgb31_preds__lgb02A_preds'),
 (650, 'difference_lgb31_preds__rnn27_preds'),
 (645, 'difference_mlp_preds__lgb31_preds'),
 (625, 'difference_lgb09_preds__lgb11A_preds'),
 (616, 'difference_mlp_preds__lgb02A_preds')

In [20]:
n_estimators = 1557
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [21]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219952
[200]	training's rmse: 0.213572
[300]	training's rmse: 0.212428
[400]	training's rmse: 0.212083
[500]	training's rmse: 0.211882
[600]	training's rmse: 0.211722
[700]	training's rmse: 0.211588
[800]	training's rmse: 0.211467
[900]	training's rmse: 0.211349
[1000]	training's rmse: 0.211238
[1100]	training's rmse: 0.21113
[1200]	training's rmse: 0.211026
[1300]	training's rmse: 0.210923
[1400]	training's rmse: 0.210825
[1500]	training's rmse: 0.210721
Did not meet early stopping. Best iteration is:
[1557]	training's rmse: 0.210662
CPU times: user 25min 32s, sys: 17.2 s, total: 25min 49s
Wall time: 3min 56s


In [22]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_1206L2.csv.gz', compression='gzip', index=False, header=True)

In [23]:
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.439387
1503425,65b9484d670f,0.131967
1503426,8bab230b2ecd,0.117773
1503427,8e348601fefc,0.072289
1503428,8bd2fe400b89,0.181978
